In [1]:
import pandas as pd
import numpy as np
from libraries.DataUtils import DataUtils
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from datetime import datetime

In [2]:
train = pd.read_csv("./data/input/train_cleaned.csv")
train = train.set_index('Id')
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,YrSold,SaleType,SaleCondition,SalePrice,HouseAge,RemodAddAge,GarageAge,RealSalePrice,SalePriceLog1p,RealSalePriceLog1p
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,5.831328,19.212182,Pave,NaN,Reg,Lvl,AllPub,Inside,...,2008,WD,Normal,208500,5,5,5,195403.106769,12.247699,12.182825
2,20,RL,6.221214,19.712205,Pave,NaN,Reg,Lvl,AllPub,FR2,...,2007,WD,Normal,181500,31,31,31,177073.170732,12.109016,12.084324
3,60,RL,5.914940,20.347241,Pave,NaN,IR1,Lvl,AllPub,Inside,...,2008,WD,Normal,223500,7,6,7,209460.884234,12.317171,12.252297
4,70,RL,5.684507,19.691553,Pave,NaN,IR1,Lvl,AllPub,Corner,...,2006,WD,Abnorml,140000,91,36,8,140000.000000,11.849405,11.849405
5,60,RL,6.314735,21.325160,Pave,NaN,IR1,Lvl,AllPub,FR2,...,2008,WD,Normal,250000,8,8,8,234296.291090,12.429220,12.364346


In [3]:
test = pd.read_csv("./data/input/test_cleaned.csv")
test = test.set_index('Id')
test.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,HouseAge,RemodAddAge,GarageAge
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,20.479373,Pave,NaN,Reg,Lvl,AllPub,Inside,...,MnPrv,NaN,0.000000,6,2010,WD,Normal,49,49,49
1462,20,RL,81.0,21.327220,Pave,NaN,IR1,Lvl,NoSewr,Corner,...,NaN,Gar2,20.777526,6,2010,WD,Normal,52,52,52
1463,60,RL,74.0,21.196905,Pave,NaN,IR1,Lvl,AllPub,Inside,...,MnPrv,NaN,0.000000,3,2010,WD,Normal,13,12,13
1464,60,RL,78.0,19.865444,Pave,NaN,IR1,Lvl,NoSewr,Inside,...,NaN,NaN,0.000000,6,2010,WD,Normal,12,12,12
1465,120,RL,43.0,17.257255,Pave,NaN,IR1,HLS,AllPub,Inside,...,NaN,NaN,0.000000,1,2010,WD,Normal,18,18,18


In [4]:
train.fillna("NA",inplace=True)
test.fillna("NA",inplace=True)
DataUtils.columns_nan_description(train)
DataUtils.columns_nan_description(test)

There are 86 columns and 0 (0.00%) of them have null values.
There are 82 columns and 0 (0.00%) of them have null values.


In [5]:
train_sale_price = train['SalePrice']
train_real_sale_price = train['RealSalePrice']
train_sale_price_log1p= train['SalePriceLog1p']
train_real_sale_price_log1p= train['RealSalePriceLog1p']
train = train.drop(['SalePrice','SalePriceLog1p','RealSalePrice','RealSalePriceLog1p'],axis=1)
train_test = pd.concat([train,test])
train_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,HouseAge,RemodAddAge,GarageAge
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,5.831328,19.212182,Pave,NA,Reg,Lvl,AllPub,Inside,...,NA,NA,0.0,2,2008,WD,Normal,5,5,5
2,20,RL,6.221214,19.712205,Pave,NA,Reg,Lvl,AllPub,FR2,...,NA,NA,0.0,5,2007,WD,Normal,31,31,31
3,60,RL,5.914940,20.347241,Pave,NA,IR1,Lvl,AllPub,Inside,...,NA,NA,0.0,9,2008,WD,Normal,7,6,7
4,70,RL,5.684507,19.691553,Pave,NA,IR1,Lvl,AllPub,Corner,...,NA,NA,0.0,2,2006,WD,Abnorml,91,36,8
5,60,RL,6.314735,21.325160,Pave,NA,IR1,Lvl,AllPub,FR2,...,NA,NA,0.0,12,2008,WD,Normal,8,8,8


In [6]:
categorical_columns = ['MSSubClass','MSZoning','Street','Alley','LotShape','LandContour',
                        'LotConfig','LandSlope','Neighborhood','Condition1','Condition2',
                        'BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd',
                        'MasVnrType','Foundation','Heating','CentralAir','Electrical','Functional',
                        'GarageType','GarageFinish','PavedDrive','Fence','MiscFeature','SaleType',
                        'SaleCondition']

label_columns = ['Utilities','OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtExposure',
                'BsmtCond','BsmtFinType1','BsmtFinType2','HeatingQC','GarageQual','KitchenQual','FireplaceQu',
                'GarageCond','PoolQC']

numerical_columns = ['LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
                        'TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath',
                        'FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt',
                        'GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
                        'PoolArea','MiscVal','MoSold','YrSold','LotFrontage']
                        
numerical_columns.extend(['HouseAge','RemodAddAge','GarageAge'])

print(f'Total columns in data set {train_test.shape[1]}')
print(f'Total columns count: {len(categorical_columns)+len(label_columns)+len(numerical_columns)}')

Total columns in data set 82
Total columns count: 82


In [7]:
#Categorical columns encoder
#handle_unknown='ignore'
OH_encoder = OneHotEncoder(drop='first', sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(train_test[categorical_columns]),columns = OH_encoder.get_feature_names(categorical_columns))
OH_cols.index = train_test.index
OH_cols.head(5)

,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [8]:
#Encode label_columns
label_columns = ['Utilities','OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtExposure',
                'BsmtCond','BsmtFinType1','BsmtFinType2','HeatingQC','GarageQual','KitchenQual','FireplaceQu',
                'GarageCond','PoolQC']
LB_cols = pd.DataFrame(train_test[label_columns])
LB_cols = LB_cols.replace(
    {'Utilities':{'AllPub':4,'NoSewr':3,'NoSeWa':2,'ELO':1},
    'ExterQual':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
    'ExterCond':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
    'BsmtQual':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0},
    'BsmtExposure':{'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0},
    'BsmtCond':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0},
    'BsmtFinType1':{'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0},
    'BsmtFinType2':{'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0},
    'HeatingQC':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
    'GarageQual':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0},
    'KitchenQual':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
    'FireplaceQu':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0},
    'GarageCond':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0},
    'PoolQC':{'Ex':4,'Gd':3,'TA':2,'Fa':1,'NA':0}}
)
MinMax_Scaler = MinMaxScaler()
LB_cols = pd.DataFrame(MinMax_Scaler.fit_transform(LB_cols),columns=label_columns)
LB_cols.index = train_test.index
LB_cols.head()

,Utilities,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtExposure,BsmtCond,BsmtFinType1,BsmtFinType2,HeatingQC,GarageQual,KitchenQual,FireplaceQu,GarageCond,PoolQC
Id,,,,,,,,,,,,,,,,
1,1.0,0.666667,0.500,0.666667,0.5,0.8,0.25,0.75,1.000000,0.166667,1.00,0.6,0.666667,0.0,0.6,0.0
2,1.0,0.555556,0.875,0.333333,0.5,0.8,1.00,0.75,0.833333,0.166667,1.00,0.6,0.333333,0.6,0.6,0.0
3,1.0,0.666667,0.500,0.666667,0.5,0.8,0.50,0.75,1.000000,0.166667,1.00,0.6,0.666667,0.6,0.6,0.0
4,1.0,0.666667,0.500,0.333333,0.5,0.6,0.25,1.00,0.833333,0.166667,0.75,0.6,0.666667,0.8,0.6,0.0
5,1.0,0.777778,0.500,0.666667,0.5,0.8,0.75,0.75,1.000000,0.166667,1.00,0.6,0.666667,0.6,0.6,0.0


In [9]:
#Numerical columns scaler
MinMax_Scaler = MinMaxScaler()
MM_cols = pd.DataFrame(MinMax_Scaler.fit_transform(train_test[numerical_columns]),columns = [numerical_columns])
MM_cols.index = train_test.index
MM_cols.head(5)

,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,LotFrontage,HouseAge,RemodAddAge,GarageAge
Id,,,,,,,,,,,,,,,,,,,,,
1,0.281320,0.949275,0.883333,0.597109,0.631368,0.0,0.509970,0.650174,0.300044,0.837086,...,0.0,0.0,0.0,0.0,0.090909,0.50,0.009685,0.043796,0.112903,0.652866
2,0.303531,0.753623,0.433333,0.000000,0.681814,0.0,0.606369,0.711321,0.436792,0.000000,...,0.0,0.0,0.0,0.0,0.363636,0.25,0.011673,0.233577,0.532258,0.735669
3,0.331740,0.934783,0.866667,0.566540,0.576542,0.0,0.675826,0.661263,0.324842,0.839838,...,0.0,0.0,0.0,0.0,0.727273,0.50,0.010111,0.058394,0.129032,0.659236
4,0.302614,0.311594,0.333333,0.000000,0.467677,0.0,0.713406,0.631350,0.339969,0.813298,...,0.0,0.0,0.0,0.0,0.090909,0.00,0.008936,0.671533,0.612903,0.662420
5,0.375179,0.927536,0.833333,0.695850,0.620109,0.0,0.696543,0.695658,0.401762,0.879003,...,0.0,0.0,0.0,0.0,1.000000,0.50,0.012150,0.065693,0.161290,0.662420


In [10]:
train_test_transformed = OH_cols.join(LB_cols)
train_test_transformed = train_test_transformed.join(MM_cols)
train_test_transformed.head(5)

,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,...,"(3SsnPorch,)","(ScreenPorch,)","(PoolArea,)","(MiscVal,)","(MoSold,)","(YrSold,)","(LotFrontage,)","(HouseAge,)","(RemodAddAge,)","(GarageAge,)"
Id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.090909,0.50,0.009685,0.043796,0.112903,0.652866
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.363636,0.25,0.011673,0.233577,0.532258,0.735669
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.727273,0.50,0.010111,0.058394,0.129032,0.659236
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.090909,0.00,0.008936,0.671533,0.612903,0.662420
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.50,0.012150,0.065693,0.161290,0.662420


In [11]:
test_data_X = train_test_transformed.loc[test.index.tolist(),:]
train_data_X = train_test_transformed.loc[train.index.tolist(),:]

In [12]:
#ElasticNet
ENet = ElasticNet()

alphas = [float(x) for x in np.linspace(start = 0, stop = 1, num = 11)]
l1_ratios = [float(x) for x in np.linspace(start = 0, stop = 1, num = 11)]
max_iters = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 100)]
tols = [1,1e-1,1e-2,1e-3,1e-4,1e-5,1e-6]

# Create the random grid
random_grid = {'alpha': alphas,
               'max_iter': max_iters,
               'tol': tols,
               'l1_ratio':l1_ratios}
print(random_grid)

{'alpha': [0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0], 'max_iter': [100, 119, 138, 157, 176, 195, 215, 234, 253, 272, 291, 311, 330, 349, 368, 387, 407, 426, 445, 464, 483, 503, 522, 541, 560, 579, 598, 618, 637, 656, 675, 694, 714, 733, 752, 771, 790, 810, 829, 848, 867, 886, 906, 925, 944, 963, 982, 1002, 1021, 1040, 1059, 1078, 1097, 1117, 1136, 1155, 1174, 1193, 1213, 1232, 1251, 1270, 1289, 1309, 1328, 1347, 1366, 1385, 1405, 1424, 1443, 1462, 1481, 1501, 1520, 1539, 1558, 1577, 1596, 1616, 1635, 1654, 1673, 1692, 1712, 1731, 1750, 1769, 1788, 1808, 1827, 1846, 1865, 1884, 1904, 1923, 1942, 1961, 1980, 2000], 'tol': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06], 'l1_ratio': [0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0]}


In [13]:
rf_random = RandomizedSearchCV(estimator = ENet, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(train_data_X, train_sale_price_log1p)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 212 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   26.0s finished


RandomizedSearchCV(cv=5, estimator=ElasticNet(), n_iter=100, n_jobs=-1,
                   param_distributions={'alpha': [0.0, 0.1, 0.2,
                                                  0.30000000000000004, 0.4, 0.5,
                                                  0.6000000000000001,
                                                  0.7000000000000001, 0.8, 0.9,
                                                  1.0],
                                        'l1_ratio': [0.0, 0.1, 0.2,
                                                     0.30000000000000004, 0.4,
                                                     0.5, 0.6000000000000001,
                                                     0.7000000000000001, 0.8,
                                                     0.9, 1.0],
                                        'max_iter': [100, 119, 138, 157, 176,
                                                     195, 215, 234, 253, 272,
                                                     

In [14]:
rf_random.best_estimator_

ElasticNet(alpha=0.0, max_iter=253, tol=1e-05)

In [15]:
#Best parameters based on random search cv
alphas = [float(x) for x in np.linspace(start = 0, stop = 0.05, num = 22)]
l1_ratios = [float(x) for x in np.linspace(start = 0, stop = 1, num = 11)]
max_iters = [int(x) for x in np.linspace(start = 150, stop = 400, num = 10)]
tols = [1e-5,1e-6,1e-7]

# Create the param grid
param_grid = {'alpha': alphas,
               'max_iter': max_iters,
               'tol': tols,
               'l1_ratio':l1_ratios}

In [16]:
grid_search = GridSearchCV(estimator = ENet, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(train_data_X, train_sale_price_log1p)

Fitting 5 folds for each of 7260 candidates, totalling 36300 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:   51.5s
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 6829 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 7922 tasks      

GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [0.0, 0.002380952380952381,
                                   0.004761904761904762, 0.0071428571428571435,
                                   0.009523809523809525, 0.011904761904761906,
                                   0.014285714285714287, 0.01666666666666667,
                                   0.01904761904761905, 0.02142857142857143,
                                   0.02380952380952381, 0.026190476190476195,
                                   0.028571428571428574, 0.030952380952380953,
                                   0.03333333333333334, 0.03571428571428572,
                                   0.0380952380952381, 0.04047619047619048,
                                   0.04285714285714286, 0.045238095238095244,
                                   0.04761904761904762, 0.05],
                         'l1_ratio': [0.0, 0.1, 0.2, 0.30000000000000004, 0.4,
                                      0.

In [17]:
best_grid = grid_search.best_estimator_
print(best_grid)

ElasticNet(alpha=0.002380952380952381, l1_ratio=0.2, max_iter=150, tol=1e-05)


In [18]:
ENet = make_pipeline(RobustScaler(),ElasticNet(alpha=0.002380952380952381, l1_ratio=0.2, max_iter=150, tol=1e-05))

ENet.fit(train_data_X, train_sale_price_log1p)

pred_test = ENet.predict(test_data_X)
pred_test = np.expm1(pred_test)

sample_submission = pd.read_csv("./data/input/sample_submission.csv")
sample_submission['SalePrice'] = pred_test
sample_submission.to_csv(f"./data/output/submission_ENet_{datetime.now().strftime('%d_%m_%Y')}.csv",index=False)